In [2]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from MLFM_GCP.circuits.QAOA import QAOA_random
from MLFM_GCP.circuits.cp_fraction import cp_fraction
from MLFM_GCP.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from MLFM_GCP.partitioning.FM.FM_main import *
from MLFM_GCP.partitioning.FM.multilevel_FM import *

def build_circuit(num_qubits,fraction,group_gates=True):
    try:
        circuit = cp_fraction(num_qubits,num_qubits,fraction)
        # circuit = QFT(num_qubits,do_swaps=False)
        # circuit = QAOA_random(num_qubits,0.5,reps=1)
        # circuit = QuantumVolume(num_qubits,num_qubits)
            
        circuit = transpile(circuit, basis_gates = ['u','cp'])
        graph = QuantumCircuitHyperGraph(circuit,group_gates=group_gates)

        return graph
    except IndexError:
        print("Conversion issue, retrying")
        return build_circuit(num_qubits,fraction,group_gates=group_gates)
    except TypeError:
        print("Conversion issue, retrying")
        return build_circuit(num_qubits,fraction,group_gates=group_gates)

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FM_CP_large_5it_2_part_0.9.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_CP_large_rec_5it_2_part_0.9.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(112, 257, 16)

# fractions = [0.3, 0.5, 0.7, 0.9]

# for fraction in fractions:
for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = 2

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    
    # Sweep the fraction parameter from 0.1 to 0.9
        # Collect data for computing means across 10 iterations
    iteration_data = []
    for iteration in range(5):
        
        # -------------------------
        # 1. Define/redefine circuit
        # -------------------------

        # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
        # circuit = QAOA_random(num_qubits,prob=0.5, reps=1)
        circuit = cp_fraction(num_qubits, num_qubits, 0.9)
        # circuit = QuantumVolume(num_qubits,num_qubits)
        circuit = transpile(circuit, basis_gates = ['u','cp'])
        base_graph = QuantumCircuitHyperGraph(circuit,group_gates=True)

        # base_graph = build_circuit(num_qubits,fraction,group_gates=True)

        depth = base_graph.depth
        initial_assignment = set_initial_partitions(qpu_info,num_qubits, depth ,num_partitions, reduced=True)
        
        # -------------------------
        # 2. Fine-grained partitioning
        # -------------------------
        # graph_list = [base_graph]
        # mapping_list = [{i : set([i]) for i in range(depth)}]
        
        # -------------------------
        # 5. Recursive refinement
        # -------------------------
        assignment_list_r, cost_list_r, time_list_r = MLFM_recursive(base_graph,
                                    initial_assignment,  
                                    qpu_info, 
                                    limit = 'qubit', 
                                    pass_list= None, 
                                    stochastic=True, 
                                    lock_nodes=False,
                                    log = False,
                                    add_initial = False,
                                    costs = None)
        
        total_time_r = sum(time_list_r)
        min_cost_r = min(cost_list_r)

        
        # -------------------------
        # 6. Store iteration-level results
        # -------------------------
        result_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "iteration": iteration,
            "r_cost":  min_cost_r,
            "time_r": total_time_r,
        }
        
        detailed_results.append(result_entry)
        iteration_data.append(result_entry)
        
        # Update detailed JSON right away
        with open(detailed_filename, "w") as f:
            json.dump(detailed_results, f, indent=2)
    
    # ---------------------------------------------------------------------
    # After 10 iterations, compute the means and log them
    # ---------------------------------------------------------------------

    r_cost_list = [x["r_cost"] for x in iteration_data]
    

    r_time_list = [x["time_r"] for x in iteration_data]
    

    mean_r_cost = float(np.mean(r_cost_list))
    

    mean_r_time = float(np.mean(r_time_list))
    
    # Print to console for quick logging
    print("=============================================")
    print(f"Finished 5 iterations for:")
    print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
    print("Mean Costs:")

    print(f"  Recursive:{mean_r_cost:.3f}")
    print("Mean Times (s):")

    print(f"  Recursive:{mean_r_time:.3f}")
    print("=============================================")
    
    # Store the aggregated means in a separate JSON
    mean_entry = {
        "num_qubits": num_qubits,
        "num_partitions": num_partitions,

        "mean_r_cost": mean_r_cost,

        "mean_r_time": mean_r_time,
    }
    
    mean_results.append(mean_entry)
    
    # Update the means JSON file
    with open(means_filename, "w") as f:
        json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Number of hypergraphs coarsened: 8
Number of hypergraphs coarsened: 8
Number of hypergraphs coarsened: 8
Number of hypergraphs coarsened: 8
Number of hypergraphs coarsened: 8
Finished 5 iterations for:
  # Qubits: 112, # Partitions: 2
Mean Costs:
  Recursive:509.400
Mean Times (s):
  Recursive:14.663
Number of hypergraphs coarsened: 8
Number of hypergraphs coarsened: 8
Number of hypergraphs coarsened: 8
Number of hypergraphs coarsened: 8
Number of hypergraphs coarsened: 8
Finished 5 iterations for:
  # Qubits: 128, # Partitions: 2
Mean Costs:
  Recursive:664.200
Mean Times (s):
  Recursive:19.853
Number of hypergraphs coarsened: 9
Number of hypergraphs coarsened: 9
Number of hypergraphs coarsened: 9
Number of hypergraphs coarsened: 9
Number of hypergraphs coarsened: 9
Finished 5 iterations for:
  # Qubits: 144, # Partitions: 2
Mean Costs:
  Recursive:850.800
Mean Times (s):
  Recursive:27.090
Number of hypergraphs coarsened: 9
Number of hypergraphs coarsened: 9
Number of hypergraphs co

KeyboardInterrupt: 

In [5]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from MLFM_GCP.circuits.QAOA import QAOA_random
from MLFM_GCP.circuits.cp_fraction import cp_fraction
from MLFM_GCP.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from MLFM_GCP.partitioning.FM.FM_main import *
from MLFM_GCP.partitioning.FM.multilevel_FM import *

def build_circuit(num_qubits,fraction,group_gates=True):
    try:
        circuit = cp_fraction(num_qubits,num_qubits,fraction)
        # circuit = QFT(num_qubits,do_swaps=False)
        # circuit = QAOA_random(num_qubits,0.5,reps=1)
        # circuit = QuantumVolume(num_qubits,num_qubits)
            
        circuit = transpile(circuit, basis_gates = ['u','cp'])
        graph = QuantumCircuitHyperGraph(circuit,group_gates=group_gates)

        return graph
    except IndexError:
        print("Conversion issue, retrying")
        return build_circuit(num_qubits,fraction,group_gates=group_gates)
    except TypeError:
        print("Conversion issue, retrying")
        return build_circuit(num_qubits,fraction,group_gates=group_gates)

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FM_QV_rec_5it_2_part.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FM_QV_rec_5it_2_part.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

# fractions = [0.3, 0.5, 0.7, 0.9]

# for fraction in fractions:
for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = 2

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    
    # Sweep the fraction parameter from 0.1 to 0.9
        # Collect data for computing means across 10 iterations
    iteration_data = []
    for iteration in range(5):
        
        # -------------------------
        # 1. Define/redefine circuit
        # -------------------------

        # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
        # circuit = QAOA_random(num_qubits,prob=0.5, reps=1)
        # circuit = QFT(num_qubits,do_swaps=False)
        # circuit = cp_fraction(num_qubits, num_qubits, 0.5)
        circuit = QuantumVolume(num_qubits,num_qubits)
        circuit = transpile(circuit, basis_gates = ['u','cp'])
        base_graph = QuantumCircuitHyperGraph(circuit,group_gates=True)

        # base_graph = build_circuit(num_qubits,fraction,group_gates=True)

        depth = base_graph.depth
        initial_assignment = set_initial_partitions(qpu_info,num_qubits, depth ,num_partitions, reduced=True)
        
        # -------------------------
        # 2. Fine-grained partitioning
        # -------------------------
        # graph_list = [base_graph]
        # mapping_list = [{i : set([i]) for i in range(depth)}]
        
        # -------------------------
        # 5. Recursive refinement
        # -------------------------
        assignment_list_r, cost_list_r, time_list_r = MLFM_recursive(base_graph,
                                    initial_assignment,  
                                    qpu_info, 
                                    limit = 'qubit', 
                                    pass_list= None, 
                                    stochastic=True, 
                                    lock_nodes=False,
                                    log = False,
                                    add_initial = False,
                                    costs = None)
        
        total_time_r = sum(time_list_r)
        min_cost_r = min(cost_list_r)

        
        # -------------------------
        # 6. Store iteration-level results
        # -------------------------
        result_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "iteration": iteration,
            "r_cost":  min_cost_r,
            "time_r": total_time_r,
        }
        
        detailed_results.append(result_entry)
        iteration_data.append(result_entry)
        
        # Update detailed JSON right away
        with open(detailed_filename, "w") as f:
            json.dump(detailed_results, f, indent=2)
    
    # ---------------------------------------------------------------------
    # After 10 iterations, compute the means and log them
    # ---------------------------------------------------------------------

    r_cost_list = [x["r_cost"] for x in iteration_data]
    

    r_time_list = [x["time_r"] for x in iteration_data]
    

    mean_r_cost = float(np.mean(r_cost_list))
    

    mean_r_time = float(np.mean(r_time_list))
    
    # Print to console for quick logging
    print("=============================================")
    print(f"Finished 5 iterations for:")
    print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
    print("Mean Costs:")

    print(f"  Recursive:{mean_r_cost:.3f}")
    print("Mean Times (s):")

    print(f"  Recursive:{mean_r_time:.3f}")
    print("=============================================")
    
    # Store the aggregated means in a separate JSON
    mean_entry = {
        "num_qubits": num_qubits,
        "num_partitions": num_partitions,

        "mean_r_cost": mean_r_cost,

        "mean_r_time": mean_r_time,
    }
    
    mean_results.append(mean_entry)
    
    # Update the means JSON file
    with open(means_filename, "w") as f:
        json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 5 iterations for:
  # Qubits: 16, # Partitions: 2
Mean Costs:
  Recursive:56.400
Mean Times (s):
  Recursive:0.822
Finished 5 iterations for:
  # Qubits: 24, # Partitions: 2
Mean Costs:
  Recursive:118.800
Mean Times (s):
  Recursive:2.201
Finished 5 iterations for:
  # Qubits: 32, # Partitions: 2
Mean Costs:
  Recursive:205.000
Mean Times (s):
  Recursive:4.258
Finished 5 iterations for:
  # Qubits: 40, # Partitions: 2
Mean Costs:
  Recursive:326.000
Mean Times (s):
  Recursive:7.189
Finished 5 iterations for:
  # Qubits: 48, # Partitions: 2
Mean Costs:
  Recursive:472.800
Mean Times (s):
  Recursive:10.752
Finished 5 iterations for:
  # Qubits: 56, # Partitions: 2
Mean Costs:
  Recursive:614.000
Mean Times (s):
  Recursive:15.537
Finished 5 iterations for:
  # Qubits: 64, # Partitions: 2
Mean Costs:
  Recursive:832.400
Mean Times (s):
  Recursive:20.370
Finished 5 iterations for:
  # Qubits: 72, # Partitions: 2
Mean Costs:
  Recursive:1027.800
Mean Times (s):
  Recursive:27.

In [ ]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from MLFM_GCP.circuits.QAOA import QAOA_random
from MLFM_GCP.circuits.cp_fraction import cp_fraction
from MLFM_GCP.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from MLFM_GCP.partitioning.FM.FM_main import *
from MLFM_GCP.partitioning.FM.multilevel_FM import *

def build_circuit(num_qubits,fraction,group_gates=True):
    try:
        circuit = cp_fraction(num_qubits,num_qubits,fraction)
        # circuit = QFT(num_qubits,do_swaps=False)
        # circuit = QAOA_random(num_qubits,0.5,reps=1)
        # circuit = QuantumVolume(num_qubits,num_qubits)
            
        circuit = transpile(circuit, basis_gates = ['u','cp'])
        graph = QuantumCircuitHyperGraph(circuit,group_gates=group_gates)

        return graph
    except IndexError:
        print("Conversion issue, retrying")
        return build_circuit(num_qubits,fraction,group_gates=group_gates)
    except TypeError:
        print("Conversion issue, retrying")
        return build_circuit(num_qubits,fraction,group_gates=group_gates)

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FM_QFT_rec_5it_3_part.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FM_QFT_rec_5it_3_part.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

# fractions = [0.3, 0.5, 0.7, 0.9]

# for fraction in fractions:
for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = 3

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    
    # Sweep the fraction parameter from 0.1 to 0.9
        # Collect data for computing means across 10 iterations
    iteration_data = []
    for iteration in range(5):
        
        # -------------------------
        # 1. Define/redefine circuit
        # -------------------------

        # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
        # circuit = QAOA_random(num_qubits,prob=0.5, reps=1)
        circuit = QFT(num_qubits,do_swaps=False)
        # circuit = cp_fraction(num_qubits, num_qubits, 0.5)
        # circuit = QuantumVolume(num_qubits,num_qubits)
        circuit = transpile(circuit, basis_gates = ['u','cp'])
        base_graph = QuantumCircuitHyperGraph(circuit,group_gates=True)

        # base_graph = build_circuit(num_qubits,fraction,group_gates=True)

        depth = base_graph.depth
        initial_assignment = set_initial_partitions(qpu_info,num_qubits, depth ,num_partitions, reduced=True)
        
        # -------------------------
        # 2. Fine-grained partitioning
        # -------------------------
        # graph_list = [base_graph]
        # mapping_list = [{i : set([i]) for i in range(depth)}]
        
        # -------------------------
        # 5. Recursive refinement
        # -------------------------
        assignment_list_r, cost_list_r, time_list_r = MLFM_recursive(base_graph,
                                    initial_assignment,  
                                    qpu_info, 
                                    limit = 'qubit', 
                                    pass_list= None, 
                                    stochastic=True, 
                                    lock_nodes=False,
                                    log = False,
                                    add_initial = False,
                                    costs = None)
        
        total_time_r = sum(time_list_r)
        min_cost_r = min(cost_list_r)

        
        # -------------------------
        # 6. Store iteration-level results
        # -------------------------
        result_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "iteration": iteration,
            "r_cost":  min_cost_r,
            "time_r": total_time_r,
        }
        
        detailed_results.append(result_entry)
        iteration_data.append(result_entry)
        
        # Update detailed JSON right away
        with open(detailed_filename, "w") as f:
            json.dump(detailed_results, f, indent=2)
    
    # ---------------------------------------------------------------------
    # After 10 iterations, compute the means and log them
    # ---------------------------------------------------------------------

    r_cost_list = [x["r_cost"] for x in iteration_data]
    

    r_time_list = [x["time_r"] for x in iteration_data]
    

    mean_r_cost = float(np.mean(r_cost_list))
    

    mean_r_time = float(np.mean(r_time_list))
    
    # Print to console for quick logging
    print("=============================================")
    print(f"Finished 5 iterations for:")
    print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
    print("Mean Costs:")

    print(f"  Recursive:{mean_r_cost:.3f}")
    print("Mean Times (s):")

    print(f"  Recursive:{mean_r_time:.3f}")
    print("=============================================")
    
    # Store the aggregated means in a separate JSON
    mean_entry = {
        "num_qubits": num_qubits,
        "num_partitions": num_partitions,

        "mean_r_cost": mean_r_cost,

        "mean_r_time": mean_r_time,
    }
    
    mean_results.append(mean_entry)
    
    # Update the means JSON file
    with open(means_filename, "w") as f:
        json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 5 iterations for:
  # Qubits: 16, # Partitions: 3
Mean Costs:
  Recursive:14.000
Mean Times (s):
  Recursive:0.466
Finished 5 iterations for:
  # Qubits: 24, # Partitions: 3
Mean Costs:
  Recursive:21.000
Mean Times (s):
  Recursive:1.356
Finished 5 iterations for:
  # Qubits: 32, # Partitions: 3
Mean Costs:
  Recursive:31.000
Mean Times (s):
  Recursive:2.709
Finished 5 iterations for:
  # Qubits: 40, # Partitions: 3
Mean Costs:
  Recursive:38.000
Mean Times (s):
  Recursive:4.979
Finished 5 iterations for:
  # Qubits: 48, # Partitions: 3
Mean Costs:
  Recursive:45.000
Mean Times (s):
  Recursive:8.307
Finished 5 iterations for:
  # Qubits: 56, # Partitions: 3
Mean Costs:
  Recursive:55.000
Mean Times (s):
  Recursive:11.937
Finished 5 iterations for:
  # Qubits: 64, # Partitions: 3
Mean Costs:
  Recursive:62.000
Mean Times (s):
  Recursive:17.179
Finished 5 iterations for:
  # Qubits: 72, # Partitions: 3
Mean Costs:
  Recursive:69.000
Mean Times (s):
  Recursive:23.126
Finis

In [ ]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from MLFM_GCP.circuits.QAOA import QAOA_random
from MLFM_GCP.circuits.cp_fraction import cp_fraction
from MLFM_GCP.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from MLFM_GCP.partitioning.FM.FM_main import *
from MLFM_GCP.partitioning.FM.multilevel_FM import *

def build_circuit(num_qubits,fraction,group_gates=True):
    try:
        circuit = cp_fraction(num_qubits,num_qubits,fraction)
        # circuit = QFT(num_qubits,do_swaps=False)
        # circuit = QAOA_random(num_qubits,0.5,reps=1)
        # circuit = QuantumVolume(num_qubits,num_qubits)
            
        circuit = transpile(circuit, basis_gates = ['u','cp'])
        graph = QuantumCircuitHyperGraph(circuit,group_gates=group_gates)

        return graph
    except IndexError:
        print("Conversion issue, retrying")
        return build_circuit(num_qubits,fraction,group_gates=group_gates)
    except TypeError:
        print("Conversion issue, retrying")
        return build_circuit(num_qubits,fraction,group_gates=group_gates)

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FM_QFT_rec_5it_4_part.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FM_QFT_rec_5it_4_part.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

# fractions = [0.3, 0.5, 0.7, 0.9]

# for fraction in fractions:
for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = 4

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    
    # Sweep the fraction parameter from 0.1 to 0.9
        # Collect data for computing means across 10 iterations
    iteration_data = []
    for iteration in range(5):
        
        # -------------------------
        # 1. Define/redefine circuit
        # -------------------------

        # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
        # circuit = QAOA_random(num_qubits,prob=0.5, reps=1)
        circuit = QFT(num_qubits,do_swaps=False)
        # circuit = cp_fraction(num_qubits, num_qubits, 0.5)
        # circuit = QuantumVolume(num_qubits,num_qubits)
        circuit = transpile(circuit, basis_gates = ['u','cp'])
        base_graph = QuantumCircuitHyperGraph(circuit,group_gates=True)

        # base_graph = build_circuit(num_qubits,fraction,group_gates=True)

        depth = base_graph.depth
        initial_assignment = set_initial_partitions(qpu_info,num_qubits, depth ,num_partitions, reduced=True)
        
        # -------------------------
        # 2. Fine-grained partitioning
        # -------------------------
        # graph_list = [base_graph]
        # mapping_list = [{i : set([i]) for i in range(depth)}]
        
        # -------------------------
        # 5. Recursive refinement
        # -------------------------
        assignment_list_r, cost_list_r, time_list_r = MLFM_recursive(base_graph,
                                    initial_assignment,  
                                    qpu_info, 
                                    limit = 'qubit', 
                                    pass_list= None, 
                                    stochastic=True, 
                                    lock_nodes=False,
                                    log = False,
                                    add_initial = False,
                                    costs = None)
        
        total_time_r = sum(time_list_r)
        min_cost_r = min(cost_list_r)

        
        # -------------------------
        # 6. Store iteration-level results
        # -------------------------
        result_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "iteration": iteration,
            "r_cost":  min_cost_r,
            "time_r": total_time_r,
        }
        
        detailed_results.append(result_entry)
        iteration_data.append(result_entry)
        
        # Update detailed JSON right away
        with open(detailed_filename, "w") as f:
            json.dump(detailed_results, f, indent=2)
    
    # ---------------------------------------------------------------------
    # After 10 iterations, compute the means and log them
    # ---------------------------------------------------------------------

    r_cost_list = [x["r_cost"] for x in iteration_data]
    

    r_time_list = [x["time_r"] for x in iteration_data]
    

    mean_r_cost = float(np.mean(r_cost_list))
    

    mean_r_time = float(np.mean(r_time_list))
    
    # Print to console for quick logging
    print("=============================================")
    print(f"Finished 5 iterations for:")
    print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
    print("Mean Costs:")

    print(f"  Recursive:{mean_r_cost:.3f}")
    print("Mean Times (s):")

    print(f"  Recursive:{mean_r_time:.3f}")
    print("=============================================")
    
    # Store the aggregated means in a separate JSON
    mean_entry = {
        "num_qubits": num_qubits,
        "num_partitions": num_partitions,

        "mean_r_cost": mean_r_cost,

        "mean_r_time": mean_r_time,
    }
    
    mean_results.append(mean_entry)
    
    # Update the means JSON file
    with open(means_filename, "w") as f:
        json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 5 iterations for:
  # Qubits: 16, # Partitions: 4
Mean Costs:
  Recursive:18.000
Mean Times (s):
  Recursive:0.634
Finished 5 iterations for:
  # Qubits: 24, # Partitions: 4
Mean Costs:
  Recursive:30.000
Mean Times (s):
  Recursive:1.905
Finished 5 iterations for:
  # Qubits: 32, # Partitions: 4
Mean Costs:
  Recursive:42.000
Mean Times (s):
  Recursive:3.937
Finished 5 iterations for:
  # Qubits: 40, # Partitions: 4
Mean Costs:
  Recursive:54.000
Mean Times (s):
  Recursive:7.698
Finished 5 iterations for:
  # Qubits: 48, # Partitions: 4
Mean Costs:
  Recursive:66.000
Mean Times (s):
  Recursive:12.150
Finished 5 iterations for:
  # Qubits: 56, # Partitions: 4
Mean Costs:
  Recursive:78.000
Mean Times (s):
  Recursive:17.412
Finished 5 iterations for:
  # Qubits: 64, # Partitions: 4
Mean Costs:
  Recursive:90.000
Mean Times (s):
  Recursive:25.247
Finished 5 iterations for:
  # Qubits: 72, # Partitions: 4
Mean Costs:
  Recursive:102.000
Mean Times (s):
  Recursive:33.783
Fin

In [2]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from MLFM_GCP.circuits.QAOA import QAOA_random
from MLFM_GCP.circuits.cp_fraction import cp_fraction
from MLFM_GCP.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from MLFM_GCP.partitioning.FM.FM_main import *
from MLFM_GCP.partitioning.FM.multilevel_FM import *

def build_circuit(num_qubits,fraction,group_gates=True):
    try:
        circuit = cp_fraction(num_qubits,num_qubits,fraction)
        # circuit = QFT(num_qubits,do_swaps=False)
        # circuit = QAOA_random(num_qubits,0.5,reps=1)
        # circuit = QuantumVolume(num_qubits,num_qubits)
            
        circuit = transpile(circuit, basis_gates = ['u','cp'])
        graph = QuantumCircuitHyperGraph(circuit,group_gates=group_gates)

        return graph
    except IndexError:
        print("Conversion issue, retrying")
        return build_circuit(num_qubits,fraction,group_gates=group_gates)
    except TypeError:
        print("Conversion issue, retrying")
        return build_circuit(num_qubits,fraction,group_gates=group_gates)

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_MLFM-R_QAOA_2_part.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_MLFM-R_QAOA_2_part.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(88, 97, 8)

# fractions = [0.3, 0.5, 0.7, 0.9]

# for fraction in fractions:
for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = 2

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    
    # Sweep the fraction parameter from 0.1 to 0.9
        # Collect data for computing means across 10 iterations
    iteration_data = []
    for iteration in range(5):
        
        # -------------------------
        # 1. Define/redefine circuit
        # -------------------------

        # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
        circuit = QAOA_random(num_qubits,prob=0.5, reps=1)
        # circuit = cp_fraction(num_qubits, num_qubits, 0.5)
        # circuit = QuantumVolume(num_qubits,num_qubits)
        circuit = transpile(circuit, basis_gates = ['u','cp'])
        base_graph = QuantumCircuitHyperGraph(circuit,group_gates=True)

        # base_graph = build_circuit(num_qubits,fraction,group_gates=True)

        depth = base_graph.depth
        initial_assignment = set_initial_partitions(qpu_info,num_qubits, depth ,num_partitions, reduced=True)
        
        # -------------------------
        # 2. Fine-grained partitioning
        # -------------------------
        # graph_list = [base_graph]
        # mapping_list = [{i : set([i]) for i in range(depth)}]
        
        # -------------------------
        # 5. Recursive refinement
        # -------------------------
        assignment_list_r, cost_list_r, time_list_r = MLFM_recursive(base_graph,
                                    initial_assignment,  
                                    qpu_info, 
                                    limit = 'qubit', 
                                    pass_list= None, 
                                    stochastic=True, 
                                    lock_nodes=False,
                                    log = False,
                                    add_initial = False,
                                    costs = None)
        
        total_time_r = sum(time_list_r)
        min_cost_r = min(cost_list_r)

        
        # -------------------------
        # 6. Store iteration-level results
        # -------------------------
        result_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "iteration": iteration,
            "r_cost":  min_cost_r,
            "time_r": total_time_r,
        }
        
        detailed_results.append(result_entry)
        iteration_data.append(result_entry)
        
        # Update detailed JSON right away
        with open(detailed_filename, "w") as f:
            json.dump(detailed_results, f, indent=2)
    
    # ---------------------------------------------------------------------
    # After 10 iterations, compute the means and log them
    # ---------------------------------------------------------------------

    r_cost_list = [x["r_cost"] for x in iteration_data]
    

    r_time_list = [x["time_r"] for x in iteration_data]
    

    mean_r_cost = float(np.mean(r_cost_list))
    

    mean_r_time = float(np.mean(r_time_list))
    
    # Print to console for quick logging
    print("=============================================")
    print(f"Finished 5 iterations for:")
    print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
    print("Mean Costs:")

    print(f"  Recursive:{mean_r_cost:.3f}")
    print("Mean Times (s):")

    print(f"  Recursive:{mean_r_time:.3f}")
    print("=============================================")
    
    # Store the aggregated means in a separate JSON
    mean_entry = {
        "num_qubits": num_qubits,
        "num_partitions": num_partitions,

        "mean_r_cost": mean_r_cost,

        "mean_r_time": mean_r_time,
    }
    
    mean_results.append(mean_entry)
    
    # Update the means JSON file
    with open(means_filename, "w") as f:
        json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 5 iterations for:
  # Qubits: 88, # Partitions: 2
Mean Costs:
  Recursive:43.000
Mean Times (s):
  Recursive:43.498
Finished 5 iterations for:
  # Qubits: 96, # Partitions: 2
Mean Costs:
  Recursive:47.000
Mean Times (s):
  Recursive:55.728
Benchmarking completed. Detailed results saved to benchmark_results_MLFM-R_QAOA_2_part.json
Aggregated means saved to benchmark_means_MLFM-R_QAOA_2_part.json
